Перед запуском убедитесь, что в корне проекта есть файл .env и в нем заполнены выданные вам креды подключения к базам данных и хранилищу

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table

In [14]:
# подгружаем .env
load_dotenv()

True

In [15]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [16]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [17]:
# Укажем таблицу для удаления
TABLE_NAME = 'alt_wb_statistic_dag'

In [18]:
# Функция удаления таблицы с помощью sqlalchemy
def delete_table(table_name, conn):
    metadata = MetaData(bind=conn)
    table = Table(table_name, metadata, autoload=True)
    table.drop(conn)

In [19]:
# Удалим таблицу
delete_table(TABLE_NAME, dst_conn)

NoSuchTableError: alt_wb_statistic_dag

In [22]:
# Проверим, что таблицы больше нет
pd.read_sql(f'select * from {TABLE_NAME}', dst_conn)

,id,project_id_alt,countryname_off,countryname,countrycode,sector1,year,year_close,totalamt,vvp,population,electricity,rural,target
0,2053,P145529,Kingdom of Morocco,Morocco,MAR,Water Supply,2014,2021,158600000,1.098814e+11,3.431808e+07,91.6,40.301,1
1,2054,P147257,Kingdom of Morocco,Morocco,MAR,Other Non-bank Financial Institutions,2014,2014,300000000,1.098814e+11,3.431808e+07,91.6,40.301,1
2,2055,P150140,Democratic Socialist Republic of Sri Lan,Sri Lanka,LKA,Other Public Administration,2014,,0,7.935930e+10,2.077100e+07,92.19206,81.68,0
3,2056,P145502,Republic of India,India,IND,Public Administration - Industry; Trade and Se...,2014,2020,200000000,2.039128e+12,1.293859e+09,81.23951,67.634,1
4,2057,P149264,Republic of India,India,IND,Capital Markets,2014,,0,2.039128e+12,1.293859e+09,81.23951,67.634,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17296,2048,P127665,Republic of Croatia,Croatia,HRV,Other Public Administration,2014,2014,206800000,5.762952e+10,4.238389e+06,100.0,41.344,1
17297,2049,P128290,Republic of the Philippines,Philippines,PHL,Mining,2014,2015,0,2.845845e+11,1.001022e+08,88.6457,55.512,1
17298,2050,P131688,Union of the Comoros,Comoros,COM,Central Government (Central Agencies),2014,2014,3800000,6.477207e+08,7.593850e+05,72.917725,71.807,1
17299,2051,P132709,Republic of Tunisia,Tunisia,TUN,ICT Infrastructure,2014,2015,250000000,4.758791e+10,1.114391e+07,99.8,33.355,1


In [10]:
# Закроем соединения в конце работы
src_conn.dispose()
dst_conn.dispose()